In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/deep-learning-from-scratch-2-master')

### Embedding 계층

In [4]:
class Embedding:
  def __init__(self,W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.idx = None

  def forward(self, idx):
    W, = self.params
    self.idx = idx
    out = W[idx]
    return out

  def backward(self, dout):
    dW, = self.grads
    dW[...] = 0
    for i, word_id in enumerate(self.idx):
      dW[word_id] += dout[i]
    return None


### Embedding Dot 계층

In [5]:
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections


class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params # 매개변수 저장
        self.grads = self.embed.grads # 기울기 저
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

### 네거티브 샘플링 - 확률분포에 따라 샘플링 예시

In [6]:
import numpy as np

# 0에서 9까지의 숫자 중 하나를 무작위로 샘플링
print(np.random.choice(10))
print(np.random.choice(10))

# words 에서 하나만 무작위로 샘플링
words = ['you', 'say', 'goodbye', 'I', ' hello', '.']
print(np.random.choice(words))

# 5개만 무작위로 샘플링(중복 있음)
print(np.random.choice(words, size = 5))

# 5개만 무작위로 샘플링(중복 없음)
print(np.random.choice(words, size = 5, replace = False))

# 확률분포에 따라 샘플링
p = [0.5,0.1,0.05,0.2,0.05,0.1]
print(np.random.choice(words, p = p))

2
5
I
['I' 'say' 'you' 'you' ' hello']
['I' 'you' ' hello' 'say' '.']
.


### 네거티브 샘플링 구현

In [7]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh


### CBOW 모델 구현 - Embedding 계층과 Negative Sampling Loss 계층을 적용해서

vocab_size : 어휘수

hidden_size : 은닉층의 뉴런 수

corpus : 단어 ID 목록

window_size : 맥락의 크기 (주변 단어 중 몇개나 맥락으로 포함시킬지)


In [8]:
from common.np import *  # import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None


In [13]:
sys.path.append('/content/drive/MyDrive/Colab Notebooks/deep-learning-from-scratch-2-master/ch04')

from common import config
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

# 하이퍼 파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
  contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
  word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
  pickle.dump(params,f, -1)

| 에폭 1 |  반복 1 / 9295 | 시간 1[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 2[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 4[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 6[s] | 손실 4.13
| 에폭 1 |  반복 81 / 9295 | 시간 8[s] | 손실 4.06
| 에폭 1 |  반복 101 / 9295 | 시간 10[s] | 손실 3.93
| 에폭 1 |  반복 121 / 9295 | 시간 11[s] | 손실 3.78
| 에폭 1 |  반복 141 / 9295 | 시간 13[s] | 손실 3.63
| 에폭 1 |  반복 161 / 9295 | 시간 15[s] | 손실 3.49
| 에폭 1 |  반복 181 / 9295 | 시간 17[s] | 손실 3.37
| 에폭 1 |  반복 201 / 9295 | 시간 18[s] | 손실 3.26
| 에폭 1 |  반복 221 / 9295 | 시간 20[s] | 손실 3.17
| 에폭 1 |  반복 241 / 9295 | 시간 21[s] | 손실 3.08
| 에폭 1 |  반복 261 / 9295 | 시간 22[s] | 손실 3.03
| 에폭 1 |  반복 281 / 9295 | 시간 24[s] | 손실 2.93
| 에폭 1 |  반복 301 / 9295 | 시간 25[s] | 손실 2.91
| 에폭 1 |  반복 321 / 9295 | 시간 28[s] | 손실 2.87
| 에폭 1 |  반복 341 / 9295 | 시간 29[s] | 손실 2.85
| 에폭 1 |  반복 361 / 9295 | 시간 31[s] | 손실 2.83
| 에폭 1 |  반복 381 / 9295 | 시간 32[s] | 손실 2.80
| 에폭 1 |  반복 401 / 9295 | 시간 34[s] | 손실 2.76
| 에폭 1 |  반복 421 / 9295 | 시간 35[s] | 손실 2.74
| 에폭 1 |  반복 441 / 92

KeyboardInterrupt: 

### CBOW 모델 평가

In [15]:
from common.util import most_similar, analogy

pkl_file = '/content/drive/MyDrive/Colab Notebooks/deep-learning-from-scratch-2-master/ch04/cbow_params.pkl'

with open(pkl_file, 'rb') as f:
  params = pickle.load(f)
  word_vecs = params['word_vecs']
  word_to_id = params['word_to_id']
  id_to_word = params['id_to_word']

# 가장 비슷한 단어 뽑기
querys = [ 'you','year','car','toyota']
for query in querys:
  most_similar(query, word_to_id, id_to_word, word_vecs, top = 5)

# 유추 작업
print('-' * 50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)



[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625
--------------------------------------------------

[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slow